In [ ]:
pip install geopy

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from geopy.geocoders import Nominatim #도로명주소를 좌표계로 변환해주는 모듈
import geopandas as gpd
import re

### 행정동코드-행정동명 맵핑1
- #### 통계청 7자리

In [ ]:
dong_df=pd.read_csv('./data/서울시_행정동_코드_통계청(7자리).csv')

In [ ]:
# 신사동 -> 신사동(관악구), 신사동(강남구)로 분리표기
for i in dong_df[(dong_df['읍면동명'] == '신사동') & (dong_df['시군구명'] == '관악구')]['읍면동명'].index:
    dong_df.at[i, '읍면동명'] = '신사동(관악구)'

for i in dong_df[(dong_df['읍면동명'] == '신사동') & (dong_df['시군구명'] == '강남구')]['읍면동명'].index:
    dong_df.at[i, '읍면동명'] = '신사동(강남구)'

In [ ]:
dong_df=dong_df.rename({'시군구명':'지역구', '읍면동명':'행정동명'},axis=1)
dong_df=dong_df[['행정동코드', '행정동명']]
dong_df['행정동명'] = dong_df['행정동명'].str.replace('일원2동', '개포3동') #일원2동 개포3동으로 변경
dong_df=dong_df.astype('str') #문자열로 타입변경
dong_df['행정동코드'] = dong_df['행정동코드'].str.replace('1123074', '1123081') #일원2동 코드를 개포3동 코드로 변경

In [ ]:
dong_df.to_csv('./data/서울시_행정동_코드_통계청(7자리)_수정.csv', index=False)

### 행정동코드-행정동명 맵핑2
- #### 행안부 8자리

In [ ]:
df_dong = pd.read_csv('./data/서울시_행정동_코드_행안부(8자리).csv')

In [ ]:
# 정규식을 사용하여 '동네이름'+'제'+'숫자'+'동'를 '동네이름'+'숫자'+'동'으로 변경
df_dong['행정동명'] = df_dong['행정동명'].apply(lambda x: re.sub(r'제(\d+)동', r'\1동', x) if '제' in x else x)

#예외 값들 별도 전처리
# 면목제3.8동 => 면목3.8동
df_dong['행정동명'] = df_dong['행정동명'].str.replace('면목제3.8동', '면목3.8동')

In [ ]:
df_dong.to_csv('./data/서울시_행정동_코드_행안부(8자리)_수정.csv')

### 카카오API

In [ ]:
temp_df=pd.read_csv('./data/추출원하는파일.csv')

In [ ]:
addr=temp_df['도로명주소']
addr

In [ ]:
locations = []
for ad in addr:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

- #### 도로명주소로 행정동명 추출

In [ ]:
town = [] ## 동
for i in range(len(locations)):

    try:
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        town.append('없음')

    except AttributeError:
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

In [ ]:
#행정동 목록을 넘파이 어레이로 받기
rct = np.array(town).T 
#행정동명을 칼럼명으로하고 위에서 받은 넘파이 어레이를 칼럼으로하는 데이터프레임 생성
hd_name = pd.DataFrame(rct, columns = ['행정동명']) 
#기존의 데이터프레임에 행정동명 칼럼 합치기
df=pd.concat([temp_df, hd_name], axis=1) 
df

- #### 도로명주소로 지역구명 추출

In [ ]:
region = [] ## 구
for i in range(len(locations)):

    try:
        region.append(locations[i][0].get('address').get('region_2depth_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        region.append('없음')

    except AttributeError:
        region.append(locations[i][0].get('road_address').get('region_2depth_name'))

In [ ]:
#지역구 목록을 넘파이 어레이로 받기
rct = np.array(region).T 
#지역구명을 칼럼명으로하고 위에서 받은 넘파이 어레이를 칼럼으로하는 데이터프레임 생성
rg_name = pd.DataFrame(rct, columns = ['행정동명']) 
#기존의 데이터프레임에 지역구명 칼럼 합치기
df=pd.concat([temp_df, rg_name], axis=1) 
df

---

In [ ]:
addr2=temp_df['지번주소']
addr2

In [ ]:
locations = []
for ad in addr2:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

- #### 지번주소로 도로명주소 추출

In [ ]:
road = [] ## 도로명주소
for i in range(len(locations)):

    try:
        road.append(locations[i][0].get('road_address').get('address_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        road.append('없음')

    except AttributeError:
        road.append(locations[i][0].get('address').get('address_name'))

In [ ]:
#도로명주소 목록을 넘파이 어레이로 받기
rct = np.array(road).T 
#도로명주소를 칼럼명으로하고 위에서 받은 넘파이 어레이를 칼럼으로하는 데이터프레임 생성
rd_name = pd.DataFrame(rct, columns = ['도로명주소']) 
#기존의 데이터프레임에 도로명주소 칼럼 합치기
df=pd.concat([temp_df, rd_name], axis=1) 
df

- #### 지번주소로 위도, 경도 추출

In [ ]:
lat = [] ## 위도
lo = [] ## 경도
for i in range(len(locations)):
    try:
        lat.append(locations[i][0].get('y'))
        lo.append(locations[i][0].get('x'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        lat.append('없음')
        lo.append('없음')

    except AttributeError:
        lat.append(locations[i][0].get('x'))
        lo.append(locations[i][0].get('y'))

In [ ]:
rct = np.array([lat,lo]).T
latlo = pd.DataFrame(rct, columns = ['위도', '경도'])
df=pd.concat([df, latlo], axis=1)
df

- #### 좌표로 행정동과 지역구 추출

In [ ]:
locations = []
for lo,la in zip(x,y):
    #좌표를 불러와서 주소 가져오기
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(lo, la) 
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API 키입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
region = [] ## 구
town = [] ## 동
for i in range(len(locations)):

    try:
        town.append(locations[i][1].get('region_3depth_name'))
        region.append(locations[i][0].get('region_2depth_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        town.append('없음')
        region.append('없음')

    except AttributeError:
        town.append(locations[i][1].get('region_3depth_name'))
        region.append(locations[i][0].get('road_address').get('region_2depth_name'))

### 전처리

- #### 채소가게 전처리

In [ ]:
store_df = pd.read_csv('./data/소상공인시장진흥공단_상가(상권)정보_서울_202306.csv')

In [ ]:
store_df = store_df[store_df['상권업종소분류코드']=='G20506'] #채소가게만 분류
store_df.reset_index(inplace=True) #인덱스 재정렬
store_df = store_df[['상호명','시도명', '시군구명', '행정동명']] #필요칼럼만 받아오기

In [ ]:
# 신사동 -> 신사동(관악구), 신사동(강남구)로 분리표기
for i in store_df[(store_df['행정동명'] == '신사동') & (store_df['시군구명'] == '관악구')]['행정동명'].index:
    store_df.at[i, '행정동명'] = '신사동(관악구)' 

for i in store_df[(store_df['행정동명'] == '신사동') & (store_df['시군구명'] == '강남구')]['행정동명'].index:
    store_df.at[i, '행정동명'] = '신사동(강남구)'

In [ ]:
store_count_df = store_df.groupby('행정동명')['상호명'].count().to_frame() #행정동별로 채소가게수 카운트
store_count_df = store_count_df.rename(columns={'상호명':'채소가게수'})
store_count_df.to_csv('./data/상가_행정동_count.csv')

- #### 제로웨이스트 전처리

In [ ]:
F1 = './data/zero.shp' 
zero = gpd.read_file(F1, encoding='cp949')

In [ ]:
zero.to_csv('./data/제로웨이스트.csv')

카카오API로 행정동명 추출과정 진행

In [ ]:
zero.to_csv('./data/서울시_제로웨이스트_행정동_위치정보.csv', index=False)

In [ ]:
zero = zero.groupby('행정동')['상호명'].count().to_frame()

In [ ]:
zero = zero.rename(columns={'상호명':'제로웨이스트 가게수'})

In [ ]:
zero.to_csv('./data/제로웨이스트 가게수.csv')

- #### 유치원 전처리

In [ ]:
kinder_df=pd.read_csv('./data/서울시 유치원 건물정보.csv', encoding='utf-8') #유치원 데이터프레임 불러오기

In [ ]:
geo_local = Nominatim(user_agent='chiricuto')

def geocoding(address): #위도, 경도 변환 함수
    try:
        geo = geo_local.geocode(address)
        x_y = [str(geo.latitude), str(geo.longitude)]
        return x_y

    except:
        return [0,0]

In [ ]:
addr = kinder_df['주소'] #유치원 도로명 주소

In [ ]:
latitude = []
longitude =[]

for i in addr: #한 번에 돌리면 시간이 너무 오래 걸려서 슬라이싱으로 적당히 나눠서 수행(ex. [:30], [30:60],...)
    latitude.append(geocoding(i)[0]) #유치원 위도 리스트 생성
    longitude.append(geocoding(i)[1]) #유치원 경도 리스트 생성

In [ ]:
kinder_df.insert(11,'위도', latitude) #유치원 데이터프레임에 위도 칼럼 만들고 위도 리스트 삽입
kinder_df.insert(12,'경도', longitude) #유치원 데이터프레임에 경도 칼럼 만들고 경도 리스트 삽입

In [ ]:
kinder_df.to_csv('./data/서울시 유치원 건물정보_위치포함.csv', index=False) #유치원 위치정보 데이터프레임 생성

In [ ]:
kinder_df=pd.read_csv('./data/서울시 유치원 건물정보_위치포함.csv', encoding='utf-8')

In [ ]:
addr=kinder_df['주소']

카카오 API로 지역구, 행정동 추출 진행

In [ ]:
# 신사동 -> 신사동(관악구), 신사동(강남구)로 분리표기
for i in kinder_df[(kinder_df['읍면동명'] == '신사동') & (kinder_df['시군구명'] == '관악구')]['읍면동명'].index:
    kinder_df.at[i, '읍면동명'] = '신사동(관악구)'

for i in kinder_df[(kinder_df['읍면동명'] == '신사동') & (kinder_df['시군구명'] == '강남구')]['읍면동명'].index:
    kinder_df.at[i, '읍면동명'] = '신사동(강남구)'

In [ ]:
kinder_df = kinder_df[['유치원명', '주소', '행정동', '위도', '경도']]

In [ ]:
kinder_df=kinder_df[~(kinder_df['행정동'] == '없음')] #결측치 제거

In [ ]:
kinder_df.to_csv('./data/서울시_유치원_행정동_위치정보.csv', index=False)

In [ ]:
g_df=kinder_df.groupby('행정동')['유치원명'].count()

In [ ]:
g_df = g_df.rename(columns={'유치원명':'유치원수'})

In [ ]:
g_df.to_csv('./data/유치원수.csv')

- #### 환경 전처리1
    -  공원 데이터

In [ ]:
filename1 = 'TB_EMNDN_CITY_FREST_RT.csv'
filepath1 = f'./data/{filename1}'

df_park = pd.read_csv(filepath1,encoding='cp949')

In [ ]:
df_park.rename(columns={'adzn_id':'행정구역코드_공원',
                        'adzn_nm':'행정동명',
                        'adzn_area':'행정구역면적',
                        'park_cnt':'공원수',
                        'park_rt':'공원율'},inplace=True)

In [ ]:
# 정규식을 사용하여 '동네이름'+'제'+'숫자'+'동'를 '동네이름'+'숫자'+'동'으로 변경
df_park['행정동명'] = df_park['행정동명'].apply(lambda x: re.sub(r'제(\d+)동', r'\1동', x) if '제' in x else x)

#예외 값들 별도 전처리
# 면목제3.8동 => 면목3.8동
df_park['행정동명'] = df_park['행정동명'].str.replace('면목제3.8동', '면목3.8동')

In [ ]:
df_park.to_csv('행정동_환경_공원.csv') 

- #### 환경 전처리2
    -  미세먼지 데이터

In [ ]:
filename2 = '거주인구_정보_(+환경지수)_211203.csv'
filepath2 = f'./data/{filename2}'

df_air = pd.read_csv(filepath2,encoding='cp949')

In [ ]:
df_air = df_air.loc[:424]

In [ ]:
df_air.rename(columns={'adstrd_cd':'행정동코드','adstrd_nm':'행정동명','frglty_psn_co':'취약자수','frglty_psn_pctl':'거주인구대비취약자백분위수','fnds_ix':'미세먼지지수','fnds_pctl':'미세먼지백분위'},inplace=True)

In [ ]:
df_air = df_air[['행정동코드','행정동명','취약자수','거주인구대비취약자백분위수','미세먼지지수','미세먼지백분위']]

In [ ]:
df_dong7 = pd.read_csv('./data/서울시_행정동_코드_통계청(7자리)_수정.csv')

In [ ]:
df_dong7.rename(columns={'행정동코드':'행정동코드7'},inplace=True)

In [ ]:
df_air_sample = df_air.merge(df_dong7, on=['행정동명'], how='outer')

In [ ]:
# "행정동명"이 "상일1동" 또는 "상일2동"인 행의 결측치를 "상일동" 행의 값으로 대체

df_air_sample.loc[df_air_sample['행정동명'].isin(['상일1동', '상일2동']), ['행정동코드', '취약자수', '거주인구대비취약자백분위수', '미세먼지지수', '미세먼지백분위']] = \
df_air_sample[df_air_sample['행정동명'] == '상일동'][['행정동코드', '취약자수', '거주인구대비취약자백분위수', '미세먼지지수', '미세먼지백분위']].values[0]


In [ ]:
df_air_sample.dropna(inplace=True)

In [ ]:
df_air_sample.reset_index(drop=True, inplace=True)

In [ ]:
df_air_sample.rename(columns={'행정동코드':'행정동코드_미세먼지'},inplace=True)
df_air_sample = df_air_sample[['행정동코드_미세먼지','행정동명','취약자수','거주인구대비취약자백분위수','미세먼지지수','미세먼지백분위','행정동코드7']]

In [ ]:
df_air_sample.to_csv('맵핑_행정동_환경_미세먼지.csv')

- #### 환경 전처리3
    -  데이터 병합

In [ ]:
df_air_park = df_air_sample.merge(df_park[['행정구역코드_공원','행정동명','행정구역면적','공원수','공원율']], on=['행정동명'], how='outer')

In [ ]:
df_air_park.to_excel('전처리_행정동_미세먼지_공원.xlsx') 

엑셀로 전처리 진행

In [ ]:
df_dong_air_park = pd.read_excel('전처리_행정동_미세먼지_공원.xlsx')
df_dong_air_park.rename(columns={'취약자수':'거주인구대비취약자수'},inplace=True)

In [ ]:
df_dong_air_park = df_dong_air_park[['행정동명','시군구명','거주인구대비취약자수','거주인구대비취약자백분위수','미세먼지지수','미세먼지백분위','행정구역면적','공원수','공원율']]

In [ ]:
df_dong_air_park.to_csv('./data/환경_미세먼지_공원.csv')

- #### 도시농업 전처리

카카오API로 도로명주소를 통해 행정동 가져와서 기존 데이터프레임에 병합해 csv파일로 저장

In [ ]:
data = pd.read_csv('./data/서울시_도시농업_행정동_위치정보.csv')

In [ ]:
# 도로명 주소에 서울이 포함된 경우만 남기기
data = data[data['도로명주소'].str.contains('서울')]
data.rename(columns={'행정동':'행정동명'}, inplace=True)

In [ ]:
# 세분화된 과정명 구분없이 프로그램 수로 카운트
program_data = data.copy()
program_data = program_data.groupby('행정동명').size().to_frame()
program_data.rename(columns={program_data.columns[0]: '도시농업(프로그램수)'}, inplace=True)

In [ ]:
# 도로명 주소로 중복된 값은 삭제하고 시설 수로 카운트 (도로명 주소가 동일한지만 양식이 다른 경우, 정확한 결과가 도출되지 않음)
filtered_program_data = data.copy()
filtered_program_data = filtered_program_data.drop_duplicates(subset='도로명주소')
filtered_program_data = filtered_program_data.groupby('행정동명').size().to_frame()
filtered_program_data.rename(columns={filtered_program_data.columns[0]: '도시농업(시설수)'}, inplace=True)

In [ ]:
# 행정동별 도시농업 프로그램수와 시설수 데이터를 합친 뒤, csv로 내보내기
merged_data = pd.concat([program_data, filtered_program_data], axis=1)
merged_data.to_csv('./data/행정동도시농업.csv')

- #### 초등학교 전처리

In [ ]:
elementary_df=pd.read_csv('./data/서울시 초등학교 기본정보.csv', encoding='cp949') #초등학교 데이터프레임 불러오기

In [ ]:
addr2 = elementary_df['도로명주소'] #초등학교 도로명 주소

In [ ]:
latitude2 = []
longitude2 =[]

for i in addr2[ : ]: #한 번에 돌리면 시간이 너무 오래 걸려서 슬라이싱으로 적당히 나눠서 수행(ex. [:30], [30:60],...)
    latitude2.append(geocoding(i)[0]) #초등학교 위도 리스트 생성
    longitude2.append(geocoding(i)[1]) #초등학교 경도 리스트 생성

In [ ]:
elementary_df.insert(9,'위도', latitude2) #초등학교 데이터프레임에 위도 칼럼 만들고 위도 리스트 삽입
elementary_df.insert(10,'경도', longitude2) #초등학교 데이터프레임에 경도 칼럼 만들고 경도 리스트 삽입

In [ ]:
elementary_df.to_csv('./data/서울시 초등학교 기본정보_위치포함.csv', index=False) #초등학교 위치정보 데이터프레임 생성

In [ ]:
elementary_df=pd.read_csv('./data/서울시 유치원 건물정보_위치포함.csv', encoding='utf-8')

In [ ]:
addr=elementary_df['주소']

카카오 API로 지역구, 행정동 추출과정을 거침

In [ ]:
# 신사동 -> 신사동(관악구), 신사동(강남구)로 분리표기
for i in elementary_df[(elementary_df['읍면동명'] == '신사동') & (elementary_df['시군구명'] == '관악구')]['읍면동명'].index:
    elementary_df.at[i, '읍면동명'] = '신사동(관악구)'

for i in elementary_df[(elementary_df['읍면동명'] == '신사동') & (elementary_df['시군구명'] == '강남구')]['읍면동명'].index:
    elementary_df.at[i, '읍면동명'] = '신사동(강남구)'

In [ ]:
elementary_df = elementary_df[['유치원명', '주소', '행정동', '위도', '경도']]

In [ ]:
elementary_df.to_csv('./data/서울시_유치원_행정동_위치정보.csv', index=False)

In [ ]:
g_df=elementary_df.groupby('행정동')['학교명'].count()

In [ ]:
g_df = g_df.rename(columns={'학교명':'초등학교수'})

In [ ]:
g_df.to_csv('./data/초등학교수.csv')

- #### 사업체 전처리 : 엑셀로 전처리 진행후 csv파일로 저장

In [ ]:
df = pd.read_excel('./data/서울시_사업체현황_행정동별.xlsx')
df=df.rename(columns={'행정구':'지역구'})

In [ ]:
df.to_csv('./data/서울시_사업체현황_행정동별.csv')

- #### 채식식당 전처리

In [ ]:
with open ("./data/채식식당.geojson", "r") as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data['features'])

In [ ]:
CONTENTS_NAME = []
ADDR_OLD = []
ADDR_NEW = []

for d in df['properties']:
   CONTENTS_NAME.append(d['CONTENTS_NAME'])
   ADDR_OLD.append(d['ADDR_OLD'])
   ADDR_NEW.append(d['ADDR_NEW'])

In [ ]:
df = pd.DataFrame({"CONTENTS_NAME":CONTENTS_NAME, "ADDR_OLD":ADDR_OLD, "ADDR_NEW":ADDR_NEW})

In [ ]:
df.to_csv('./data/채식식당.csv', index=False)

카카오 API로 행정동과 지역구 추출하여 신사동 신사동(강남구)와 신사동(관악구)로 분리표기 작업 진행

In [ ]:
df #상호명, 도로명주소, 행정동만 남긴 데이터프레임

In [ ]:
g_df=df.groupby('행정동')['상호명'].count()

In [ ]:
g_df = g_df.rename(columns={'상호명':'채식식당수'})

In [ ]:
g_df.to_csv('./data/채식식당수.csv')

- #### 카페 전처리

In [ ]:
from pandas.io.json import json_normalize

with open('./data/서울시 휴게음식점 인허가 정보.json') as f:
    d= json.load(f)

In [ ]:
df = json_normalize(d, 'DATA')

In [ ]:
df=df[df['dtlstatenm']=='영업'] #영업점만 남기기
df=df[(df['uptaenm']=='커피숍')|(df['uptaenm']=='기타 휴게음식점')|(df['uptaenm']=='키즈카페')] #최대한 카페만 가져오기 위한 항목들만 선택
df=df[~(df['bplcnm'].str.contains('스타벅스'))] #직영점비율이 높은 별다방 제외
df=df.reset_index(drop=True) #인덱스 재정렬

In [ ]:
df=df[['bplcnm','rdnwhladdr']] #상호명과 도로명주소칼럼만 가져옴
df['수정주소'] = df['rdnwhladdr'] #위도 경도를 구하기 위해 정제된 도로명주소 칼럼을 별도로 생성
df=df.astype({'수정주소':'str'}) #수정주소 타입을 문자열로 변경

In [ ]:
df['rdnwhladdr'].replace('', np.nan, inplace=True) #도로명주소 빈 값 nan값으로 대체
df.dropna(subset=['rdnwhladdr'], inplace=True) #결측치 삭제

In [ ]:
df=df.reset_index(drop=True) #인덱스 재정렬

In [ ]:
addr = df['수정주소']
for i in range(len(addr)):
    a = addr[i].split(',')
    addr[i] = a[0] #콤마 뒤 부속주소 삭제
print(addr)

In [ ]:
for i in range(len(addr)):
    a = addr[i].split('(')
    addr[i] = a[0] #추가로 남은 괄호 뒷부분 삭제
print(addr)

In [ ]:
from tqdm import tqdm

In [ ]:
locations = []
for ad in tqdm(addr[:7000]):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
file_path = "./data/cafe.start_7000.json"
# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(locations, json_file)
print(f"list -> json 파일로 내보내기 완료 {file_path}")

In [ ]:
locations = []
for ad in tqdm(addr[7000:10000]):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
file_path = "./data/cafe.7000_10000.json"
# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(locations, json_file)
print(f"list -> json 파일로 내보내기 완료 {file_path}")

In [ ]:
locations = []
for ad in tqdm(addr[10000:15000]):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
file_path = "./data/cafe.10000_15000.json"
# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(locations, json_file)
print(f"list -> json 파일로 내보내기 완료 {file_path}")

In [ ]:
locations = []
for ad in tqdm(addr[15000:22265]):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(ad)
    headers = {
    ## "Authorization": "KakaoAK 뒤에 본인의 REST API키 입력"
    "Authorization": "KakaoAK"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
file_path = "./data/cafe.15000_end.json"
# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(locations, json_file)
print(f"list -> json 파일로 내보내기 완료 {file_path}")

In [ ]:
with open('./data/cafe_start_7000.json', encoding = 'utf-8') as f:
  pt1 = json.load(f)

with open('./data/cafe_7000_10000.json', encoding = 'utf-8') as f:
  pt2 = json.load(f)

with open('./data/cafe_10000_15000.json', encoding = 'utf-8') as f:
  pt3 = json.load(f)

with open('./data/cafe_15000_end.json', encoding = 'utf-8') as f:
  pt4 = json.load(f)

list=pt1+pt2+pt3+pt4

카카오API를 활용하여 행정동과 지역구, 위도와 경도 정보를 받아서 데이터프레임에 칼럼으로 붙임

In [ ]:
df['행정동명'].replace('없음', np.nan, inplace=True) 
df.dropna(subset=['행정동명'], inplace=True) #행정동명이 없는 곳은 결측치로보고 삭제
df=df.reset_index(drop=True)

In [ ]:
df.rename({'bplcnm':'상호명','rdnwhladdr':'도로명주소'}, axis=1, inplace=True) 

In [ ]:
df=df[['상호명','도로명주소','행정동명','위도','경도']] #필요 칼럼만 남기기

In [ ]:
# 신사동 -> 신사동(관악구), 신사동(강남구)로 분리표기
for i in store_df[(store_df['행정동명'] == '신사동') & (store_df['시군구명'] == '관악구')]['행정동명'].index:
    store_df.at[i, '행정동명'] = '신사동(관악구)' 

for i in store_df[(store_df['행정동명'] == '신사동') & (store_df['시군구명'] == '강남구')]['행정동명'].index:
    store_df.at[i, '행정동명'] = '신사동(강남구)'

In [ ]:
df.to_csv('./data/서울시_카페_위치정보.csv', index=False)

In [ ]:
g_df=df.groupby('행정동명')['상호명'].count().to_frame()

In [ ]:
g_df = g_df.rename(columns={'상호명':'카페수'})

In [ ]:
g_df.to_csv('./data/카페 수.csv')

- #### 유동인구 전처리

    -  서울시 지하철호선별 역별 승하차인원 엑셀로 집계하여 주소정보와 병합후 코드로 병합 진행

In [ ]:
df=pd.read_csv('./data/서울지하철_공공데이터_호선별_역별_승하차인원_일평균.csv')

In [ ]:
df=df.sort_values(['호선','역명'])
df=df.reset_index(drop=True)

In [ ]:
df2=pd.read_csv('./data/서울시_지하철호선별_역별_위치정보.csv')

행정동이 결측치인 것은 엑셀로 직접 찾아 수정함

In [ ]:
m_df=pd.merge(df,df2, on=['호선', '역명'], how='left')

In [ ]:
m_df.to_csv('./data/서울지하철_공공데이터_호선별_역별_승하차인원_일평균_행정동포함.csv', index=False)

- #### 유동인구 전처리2  
    - 버스 전처리

In [ ]:
bus_df = pd.read_csv('./data/시간대별_승하차_인원_정보_버스.csv', encoding='cp949')

In [ ]:
bus_df['합계'] = bus_df['승차']+bus_df['하차']

In [ ]:
bus_df = bus_df[['표준버스정류장ID','버스정류장ARS번호', '역명', '합계']]

In [ ]:
bus_df.to_csv('./data/버스합계.csv', index=False)

In [ ]:
bus_location_df = pd.read_csv('./data/서울시버스정류소ID_위도_경도.csv')

카카오API로 좌표를 통해 지역구와 행정동 추출하는 과정 진행

In [ ]:
bus_location_df #행정동, 지역구 포함된 데이터프레임

In [ ]:
for i in bus_location_df[(bus_location_df['행정동'] == '신사동') & (bus_location_df['구'] == '관악구')]['행정동'].index:
    bus_location_df.at[i, '행정동'] = '신사동(관악구)'

for i in bus_location_df[(bus_location_df['행정동'] == '신사동') & (bus_location_df['구'] == '강남구')]['행정동'].index:
    bus_location_df.at[i, '행정동'] = '신사동(강남구)'

In [ ]:
bus_location_df['행정동'] = bus_location_df['행정동'].apply(lambda x: re.sub(r'제(\d+)동', r'\1동', x) if '제' in x else x)

In [ ]:
bus_location_df.to_csv('./data/버스정류소 행정동.csv')

In [ ]:
merge_df = pd.merge(bus_df, bus_location_df, left_on = '표준버스정류장ID', right_on = 'NODE_ID', how='left')

In [ ]:
merge_df.to_csv("./data/버스 합계 행정동별.csv")

In [ ]:
merge_df = merge_df.dropna(axis=0)

In [ ]:
merge_df = merge_df.groupby(['행정동'])['합계'].sum().to_frame()
merge_df.reset_index(inplace=True)

In [ ]:
merge_df.to_csv('./data/버스 행정동별 합계 최종.csv')

- #### 유동인구 전처리3  
    - 지하철 승하차인원과 버스승하차인원 정보를 병합하여 유동인구 데이터 생성

In [ ]:
subway_data = pd.read_csv('./data_to_preprocess/서울지하철_호선별_역별_승하차인원_일평균_행정동포함.csv', usecols = ['행정동명', '승하차인원'])
bus_data = pd.read_csv('./data_to_preprocess/버스 행정동별 합계 최종.csv', index_col=0)

In [ ]:
# 소수점 값이 포함된 지하철 승하차 인원은 반올림 후, 정수로 형 변화
subway_data['승하차인원'] = round(subway_data['승하차인원']).astype('int')

In [ ]:
# 행정동별로 그룹핑하고 합계 내기
subway_data = subway_data.groupby('행정동명').sum().reset_index()

In [ ]:
# 버스와 지하철 데이터 합치기
final_df = pd.merge(bus_data, subway_data, on=['행정동명'], how='left', suffixes=['_버스', '_지하철'])
final_df.fillna(0, inplace=True)

In [ ]:
# 승하차 인원 합계
final_df['승하차인원_버스_지하철'] = final_df['승하차인원_버스'] + final_df['승하차인원_지하철']
final_df=final_df['행정동명', '승하차인원_버스_지하철']

In [ ]:
final_df.to_csv('./data/버스_지하철_승하차인원_8월평균.csv', index=False)

- #### 매출 전처리

In [ ]:
data_money =pd.read_csv('./data/서울시_상권분석서비스(상권_추정매출)_2022년_복사본.csv',encoding='cp949')

### 병합

- #### 행정동명 붙인 파일들 불러오기

In [ ]:
store_count_df = pd.read_csv('./data/상가_행정동_count.csv', index_col=0) #채소가게 수
generation_df = pd.read_csv('./data/세대원 구성 426.csv', index_col=0) #세대구성별 세대수
zero_df = pd.read_csv('./data/제로웨이스트 가게수.csv', index_col=0) #제로웨이스트 가게수
kinder_df = pd.read_csv('./data/유치원수.csv', index_col=0) #유치원 수
eco_df = pd.read_csv('./data/환경_미세먼지_공원.csv', index_col=0) #거주인구대비취약자수 및 백분위, 미세먼지지수 및 백분위, 행정구역면적, 공원수, 공원율
farm_df = pd.read_csv('./data/행정동도시농업.csv', index_col=0) #도시농업 프로그램 및 시설 수
elementary_df = pd.read_csv('./data/초등학교수.csv', index_col=0) #초등학교 수
business_df = pd.read_csv('./data/사업체및종사자수.csv', index_col=0) #사업체 및 종사자 수
vegan_df = pd.read_csv('./data/채식식당수.csv', index_col=0) #채식식당 수
people_df = pd.read_csv('./data/거주인구수.csv', index_col=0) #인구 수
age_gender_df = pd.read_csv('./data/행정동별 성별 연령대별 거주인구.csv') #성별과 연령대별 인구 수
cafe_df = pd.read_csv('./data/카페 수.csv', index_col=0) #카페 수
move_df = pd.read_csv('./data/버스_지하철_승하차인원_8월평균.csv', index_col=0) #유동인구 수
sales_df = pd.read_csv('./data/행정동_매출액.csv', encoding='cp949', index_col=0) #점포당 1년 매출액
year_sales_df = pd.read_csv('./data/행정동별_1년전체매출액_수정.csv', encoding='cp949', index_col=0) #1년 전체 매출액

- #### 데이터프레임 병합

In [ ]:
m_df_0 = pd.merge(generation_df, store_count_df, left_index=True, right_index=True, how='outer')
m_df_1 = pd.merge(m_df_0, zero_df, left_index=True, right_index=True, how='outer')
m_df_2 = pd.merge(m_df_1, kinder_df, left_index=True, right_index=True, how='outer')
m_df_3 = pd.merge(m_df_2, eco_df, left_index=True, right_index=True, how='outer')
m_df_4 = pd.merge(m_df_3, farm_df, left_index=True, right_index=True, how='outer')
m_df_5 = pd.merge(m_df_4, elementary_df, left_index=True, right_index=True, how='outer')
m_df_6 = pd.merge(m_df_5, business_df, left_index=True, right_index=True, how='outer')
m_df_7 = pd.merge(m_df_6, vegan_df, left_index=True, right_index=True, how='outer')
m_df_8 = pd.merge(m_df_7, people_df, left_index=True, right_index=True, how='outer')
m_df_9 = pd.merge(m_df_8, age_gender_df, left_index=True, right_index=True, how='outer')
m_df_10 = pd.merge(m_df_9, cafe_df, left_index=True, right_index=True, how='outer')
m_df_11 = pd.merge(m_df_10, move_df, left_index=True, right_index=True, how='outer')
m_df_12 = pd.merge(m_df_11, sales_df, left_index=True, right_index=True, how='outer')
m_df_13 = pd.merge(m_df_12.reset_index().rename(columns={'index': '행정동명'}), year_sales_df, on='행정동명', how='left')

- #### 병합된 데이터프레임 정제 후 저장

In [ ]:
m_df_13.fillna(0, inplace=True) #결측치 0으로 대체
m_df_13.set_index('행정동명', drop=True, inplace=True)
m_df_13.to_csv('./data/final_data_before_winsorization.csv')

- #### 저장된 데이터셋 확인

In [15]:
data=pd.read_csv('./data/final_data_before_winsorization.csv',index_col=0)
print(f'사용하는 컬럼수: {len(data.columns)}개')

사용하는 컬럼수: 50개
